In [1]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objs as go
from pathlib import Path

# 1. Find all CSVs (ports) in the folder
DATA_DIR = Path(".")  # Change this if your CSVs are in another folder
csv_files = list(DATA_DIR.glob("*.csv"))
ports = [f.stem for f in csv_files]
port_file_map = dict(zip(ports, csv_files))

# Helper: Load data for a port
def load_port_data(port):
    df = pd.read_csv(
        port_file_map[port],
        usecols=['date_time_utc', 'mmsi', 'latitude', 'longitude', 'ship_name', 'length', 'ship_type'],
        parse_dates=['date_time_utc'],
        dayfirst=True,
        low_memory=False
    )
    df['ship_type'] = df['ship_type'].astype(str)
    return df

# Preload the first port for initial filter settings
initial_df = load_port_data(ports[0])
initial_types = sorted(initial_df['ship_type'].unique())
initial_min_len, initial_max_len = int(initial_df['length'].min()), int(initial_df['length'].max())
initial_vessels = initial_df[['mmsi', 'ship_name']].drop_duplicates()
initial_vessel_opts = [
    {"label": f"{row.ship_name} ({row.mmsi})", "value": row.mmsi}
    for _, row in initial_vessels.iterrows()
]
initial_vessel_count = f"{len(initial_vessels)} vessels found."

# --- LAYOUT ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "AIS Port Vessel Dashboard"

app.layout = dbc.Container([
    html.H1("Port AIS Vessel Explorer", className="my-3"),
    dbc.Row([
        dbc.Col([
            html.Label("Port:", className="mt-2"),
            dcc.Dropdown(
                id="port-dropdown",
                options=[{"label": p, "value": p} for p in ports],
                value=ports[0], clearable=False
            ),
            html.Label("Ship Type:", className="mt-3"),
            dcc.Dropdown(
                id="shiptype-dropdown",
                options=[{"label": t, "value": t} for t in initial_types],
                multi=True,
                placeholder="Select ship type(s)",
                value=[]
            ),
            html.Label("Length Range (meters):", className="mt-3"),
            dcc.RangeSlider(
                id="length-slider",
                min=initial_min_len,
                max=initial_max_len,
                step=1,
                value=[initial_min_len, initial_max_len],
                marks={i: str(i) for i in range(initial_min_len, initial_max_len+1, 50)}
            ),
            html.Label("Vessels:", className="mt-3"),
            dcc.Dropdown(
                id="vessel-dropdown",
                options=initial_vessel_opts,
                multi=True,
                placeholder="Select vessel(s)",
                value=[]
            ),
            html.Div(initial_vessel_count, id="vessel-count", className="my-2 text-muted"),
        ], md=3),
        dbc.Col([
            dcc.Graph(id="map-graph", style={"height": "80vh"}),
        ], md=9)
    ])
], fluid=True)

# --- SINGLE COMBINED CALLBACK FOR FILTERS ---
@app.callback(
    Output("shiptype-dropdown", "options"),
    Output("shiptype-dropdown", "value"),
    Output("length-slider", "min"),
    Output("length-slider", "max"),
    Output("length-slider", "value"),
    Output("length-slider", "marks"),
    Output("vessel-dropdown", "options"),
    Output("vessel-dropdown", "value"),
    Output("vessel-count", "children"),
    Input("port-dropdown", "value"),
    Input("shiptype-dropdown", "value"),
    Input("length-slider", "value"),
)
def update_all_filters(port, selected_types, length_range):
    df = load_port_data(port)
    # Ship type options
    types = sorted(df['ship_type'].unique())
    shiptype_opts = [{"label": t, "value": t} for t in types]

    # Set up selected_types and length_range defaults
    if selected_types is None:
        selected_types = []
    if length_range is None or len(length_range) != 2:
        min_len, max_len = int(df['length'].min()), int(df['length'].max())
        length_range = [min_len, max_len]
    min_len, max_len = int(df['length'].min()), int(df['length'].max())
    marks = {i: str(i) for i in range(min_len, max_len+1, 50)}
    # Filter data
    filtered_df = df.copy()
    if selected_types:
        filtered_df = filtered_df[filtered_df['ship_type'].isin(selected_types)]
    filtered_df = filtered_df[filtered_df['length'].between(length_range[0], length_range[1])]
    vessel_df = filtered_df[['mmsi', 'ship_name']].drop_duplicates()
    vessel_opts = [
        {"label": f"{row.ship_name} ({row.mmsi})", "value": row.mmsi}
        for _, row in vessel_df.iterrows()
    ]
    vessel_count = f"{len(vessel_df)} vessels found."
    return (
        shiptype_opts, selected_types,
        min_len, max_len, [length_range[0], length_range[1]], marks,
        vessel_opts, [],
        vessel_count
    )

# --- MAP CALLBACK ---
@app.callback(
    Output("map-graph", "figure"),
    Input("port-dropdown", "value"),
    Input("shiptype-dropdown", "value"),
    Input("length-slider", "value"),
    Input("vessel-dropdown", "value"),
)
def plot_map(port, selected_types, length_range, selected_vessels):
    df = load_port_data(port)
    # Apply filters
    if selected_types:
        df = df[df['ship_type'].isin(selected_types)]
    if length_range:
        df = df[df['length'].between(length_range[0], length_range[1])]
    if not selected_vessels:
        # Empty map
        return go.Figure()
    fig = go.Figure()
    colors = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]
    for idx, mmsi in enumerate(selected_vessels):
        sub = df[df['mmsi'] == mmsi].sort_values('date_time_utc')
        if sub.empty:
            continue
        color = colors[idx % len(colors)]
        # Path as scattermapbox line, each point clickable for timestamp
        fig.add_trace(go.Scattermapbox(
            lat=sub['latitude'], lon=sub['longitude'],
            mode="markers+lines",
            marker=dict(size=7),
            line=dict(width=2),
            name=f"{sub['ship_name'].iloc[0]} ({mmsi})",
            text=sub['date_time_utc'].astype(str),
            hoverinfo="text+name",
            customdata=sub['date_time_utc'].astype(str),
            marker_color=color,
        ))
        # Start marker
        fig.add_trace(go.Scattermapbox(
            lat=[sub['latitude'].iloc[0]], lon=[sub['longitude'].iloc[0]],
            mode="markers", marker=dict(size=13, color="lime"),
            name=f"Start: {sub['date_time_utc'].iloc[0]}",
            text=["Start"],
            hoverinfo="text+name",
        ))
        # End marker
        fig.add_trace(go.Scattermapbox(
            lat=[sub['latitude'].iloc[-1]], lon=[sub['longitude'].iloc[-1]],
            mode="markers", marker=dict(size=13, color="red"),
            name=f"End: {sub['date_time_utc'].iloc[-1]}",
            text=["End"],
            hoverinfo="text+name",
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=12,
        mapbox_center={"lat": df['latitude'].mean(), "lon": df['longitude'].mean()},
        margin={"r":0,"t":0,"l":0,"b":0},
        legend=dict(orientation="h")
    )
    return fig

# --- RUN APP ---
if __name__ == "__main__":
    app.run_server(debug=True, port=8040)
